# Motion Magnification using 2D DTCWT

We use python dtcwt module for the decomposition part.

In [1]:
pip install dtcwt

     |████████████████████████████████| 70 kB 3.2 MB/s 
  Created wheel for dtcwt: filename=dtcwt-0.12.0-py3-none-any.whl size=87884 sha256=3eac2765627a513a4aa9c00cfec047690972a91150d32599bf0ee3eeb6401465
  Stored in directory: /root/.cache/pip/wheels/36/17/44/41d78d570172185f944f9b79f983905ab9072d70dbdb118710
Successfully built dtcwt


In [2]:
import dtcwt
from scipy import ndimage
from scipy import signal
import numpy as np
import cv2
import os
import requests


def phase(x):
    ind = np.where(np.abs(x) > 1e-20)
    ph = np.ones_like(x)
    ph[ind] = x[ind]/np.abs(x[ind])
    return(ph)

def get_phases(pyramids,level):
    sz = pyramids[0].highpasses[level].size
    length = len(pyramids)
    ph = np.empty((length,sz),pyramids[0].highpasses[level].dtype)
    ph[0,:] = phase(pyramids[0].highpasses[level].flatten())
    ph_prev = ph[0,:]
    for i in range(1,length):
        ph_cur = phase(pyramids[i].highpasses[level].flatten())
        ph[i,:] = ph_cur / ph_prev
        ph_prev = ph_cur
    ang = np.angle(ph)
    ang =np.cumsum(ang, axis =0)
    return(ang)

def flattop_filter1d(data,width, axis = 0, mode = 'reflect'):
    window_size =  round(width/0.2327)
    window = signal.flattop(window_size)
    window = window/np.sum(window)
    result = ndimage.convolve1d(data, window, axis = axis, mode = 'reflect')
    return(result)

def magnify_motions_2d(data, k = 8., width = 70):
    nlevels = 8
    tr = dtcwt.Transform2d()
    pyramids = []
    n = np.shape(data)[0]
    #print('Forward DTCWT...', end = ' ', flush = 1)
    for i in range(0,n):
        pyramids.append(tr.forward(data[i,:,:],nlevels =nlevels))
    #print('DONE')
    #print('Modifying phase...', end = ' ', flush = 1)
    for level in range(0,nlevels):
        phase = get_phases(pyramids,level)
        phase0 = flattop_filter1d(phase,width, axis = 0, mode = 'reflect')
        phase = phase0 + (phase - phase0)*k
        phase =  flattop_filter1d(phase,2.0, axis = 0, mode = 'reflect')
        for i in range(0,n):
            h = pyramids[i].highpasses[level]
            abs_value = np.abs(h).flatten()
            h = abs_value * np.exp(1j*phase[i,:])
            pyramids[i].highpasses[level][:] = np.reshape(h,np.shape(pyramids[i].highpasses[level][:]))
    result = np.empty_like(data)
    #print('DONE')
    #print('Inverse DTCWT...', end = ' ', flush = 1)
    for i in range(0,n):
        result[i,:,:] = tr.inverse(pyramids[i])
    #print('DONE')
    return(result)

## Downloading the input video to our notebook
To do a demo of our motion magnification algorithm we use a video which was used in the original paper. </br>
You can alternatively use you own video as input too. You will have to upload the video to the colab notebook, and rename the 'filename' variable as the name of your video.</br>
If you want to try your own video, comment the first line in the next code block and Uncomment the next line and enter your own filename.


In [3]:
filename = "video.mp4" # If you want to try your own video, comment this line
                      # and Uncomment the next line and enter your own filename

#filename=""


##helper function for downloading video
You can skip the downloading part in this notebook if you are using your own video. 

In [4]:
def download_file(url, dest_filename):
    """Downloads the file in given url"""
    if os.path.isfile(dest_filename):
        print('Already Downloaded: %s to %s' % (url, dest_filename))
        return
    print('Downloading: %s to %s' % (url, dest_filename))

    response = requests.get(url, stream=True)
    if not response.ok:
        raise Exception("Couldn't download file")

    with open(dest_filename, 'wb') as fp:
        for block in response.iter_content(1024):
            fp.write(block)

## Downloading....
Skip the next code block if you are using your own video.

In [5]:
download_file('http://people.csail.mit.edu/mrub/evm/video/face.mp4',filename)

Downloading: http://people.csail.mit.edu/mrub/evm/video/face.mp4 to video.mp4


##Running the motion magnification

In [ ]:
cap = cv2.VideoCapture(filename)

fps = cap.get(cv2.CAP_PROP_FPS)
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

input_data_r = np.empty((frameCount, frameHeight, frameWidth), np.dtype('double'))
input_data_g = np.empty((frameCount, frameHeight, frameWidth), np.dtype('double'))
input_data_b = np.empty((frameCount, frameHeight, frameWidth), np.dtype('double'))

fc = 0
ret = True

while (fc < frameCount  and ret):
    ret, frame = cap.read()
    input_data_b[fc]=frame[:,:,0]
    input_data_g[fc]=frame[:,:,1]
    input_data_r[fc]=frame[:,:,2]
    fc += 1

cap.release()

print("Loaded video into numpy arrays...")

# mag_range=[3,5,7]
# for k in mag_range:


k= 3 #Magnification
width= 80 # width



result_r=magnify_motions_2d(input_data_r,k,width)
result_uint8_r = result_r.astype(np.uint8)
print('red channel done')
result_g=magnify_motions_2d(input_data_g,k,width)
result_uint8_g = result_g.astype(np.uint8)
print('green channel done')
result_b=magnify_motions_2d(input_data_b,k,width)
result_uint8_b = result_b.astype(np.uint8)
print('blue channel done')
print('creating an empty numpy array...')
result_uint8 = np.empty((frameCount,frameHeight, frameWidth,3), dtype=np.uint8)

fc=0
while (fc < frameCount):

    result_uint8[fc][:,:,0] = result_uint8_b[fc]
    result_uint8[fc][:,:,1] = result_uint8_g[fc]
    result_uint8[fc][:,:,2] = result_uint8_r[fc]
    fc += 1

print('copied channels to a single data file')

################# Save the Movie as mp4 ###############

fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter(("mag_k"+str(k)+"_"+filename),fourcc, fps, (frameWidth,frameHeight))
  
# running the loop 

fc = 0

while (fc < frameCount):
    frame = result_uint8[fc]
    out.write(frame)
    fc += 1

out.release() 
print("Saved to output video file.")